# Example Notebook

In [1]:
import json 
from pathlib import Path
import shutil

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.data_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


PROJECT_DIRECTORY = /Users/heslinge/Repos/Wattile/wattile


In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_output"]["exp_dir"] = str(exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': '/Users/heslinge/Repos/Wattile/tests/data/Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power'},
 'data_output': {'exp_dir': '/Users/heslinge/Repos/Wattile/notebooks/exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'feat_time': {'month_of_year': ['sincos'],
   'day_of_week': ['binary_reg', 'binary_fuzzy'],
   'hour_of_d

In [3]:
"""
Firstly, we will read the raw data from the dataset. 
Checkout the docs for an indepth explaination of necessary dataset structure.
"""
data = read_dataset_from_file(configs)
data


,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


In [4]:
"""
Next, we call `prep_for_rnn` do preform some data preprocessing.
"""
train_df, val_df = prep_for_rnn(configs, data)
train_df

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
0,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.0,0.0,0.497222,0.502778,0.0,0.0,0.0,-0.493776,0.869589,99.349998
1,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.0,0.0,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,99.800003
2,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.0,0.0,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.900002
3,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.0,0.0,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,101.500000
4,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.0,0.0,0.455556,0.544444,0.0,0.0,0.0,-0.493776,0.869589,98.504036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,15.582798,13.926300,0.738270,48.812000,13.456160,22.980000,11.688332,-0.254602,0.967046,0.0,...,0.0,0.0,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,107.037003
515,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.0,0.0,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,109.573402
516,16.985941,-0.961478,-0.831852,47.714001,-1.329346,26.340000,16.777510,0.004363,0.997859,1.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,102.709602
517,16.848026,-1.045253,-0.634286,47.588001,-1.507714,26.610001,11.603326,0.069756,0.991445,1.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [5]:
"""
Finally, we are ready to train our model!
"""
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)
model.train(train_df, val_df)

Logging to: /Users/heslinge/Repos/Wattile/notebooks/exp_dir/output.out, PID: 45540
saving timeseries comparison in /Users/heslinge/Repos/Wattile/notebooks/exp_dir/Vis_TimeseriesComparisons.svg


In [6]:
f"""
See {exp_dir} for the results.
"""

'\nSee /Users/heslinge/Repos/Wattile/notebooks/exp_dir for the results.\n'